Notebook Setup
---

In [8]:
import os
import sys

In [9]:
%load_ext autoreload
#Now, you can turn on auto-reloading
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Tasks
---

You need to build a schedule for 4 courses:
1. *Practical Programming Methodology* (2 lec, 2 labs per week)
2. *Algorithms I* (2 lec, 1 lab per week)
3. *Operating Systems* (2 lec, 1 lab per week)
4. *Introduction to File and Database Management* (2 lec, 1 lab per week)

Courses could be on *Monday, Tuesday, Wednesday, Thursday* and *Friday*.

Each day of week has 3 time slots (*class1, class2, class3*)

There can't be 2 lectures on the same course on the same day, but there can be a lecture and a lab on the same course on the same day.

For example:

```
Wrong case:
* Mon, class1: Practical Programming Methodology - lec
* Mon, class2: Algorithms I - lec
* Mon, class3: Practical Programming Methodology - lec

Correct case:
* Mon, class1: Practical Programming Methodology - lec
* Mon, class2: Algorithms I - lec
* Mon, class3: Practical Programming Methodology - lab
```

There can't be 2 lectures on the same course on an adjacent day, but there can be a lecture and a lab on the same course on an adjacent day.
```
Wrong case:
* Mon, class1: Practical Programming Methodology - lec
* Mon, class2: Algorithms I - lec
* Tue, class1: Practical Programming Methodology - lec

Correct case:
* Mon, class1: Practical Programming Methodology - lec
* Mon, class2: Algorithms I - lec
* Tue, class1: Practical Programming Methodology - lab
```

There can't be 2 labs on the same course on an adjacent day.
```
Wrong case:
* Mon, class1: Practical Programming Methodology - lab
* Mon, class2: Algorithms I - lec
* Tue, class1: Practical Programming Methodology - lab
```

Task 1 - Formulating the Problem as a CSP Problem

In [10]:
from utils import parse_neighbors

days = ['Mon','Tue','Wed','Thu','Fri']
slots = ['Class1','Class2','Class3']
timeSlots = [d + s for d in days for s in slots]

relationships = ''
for ts1 in range(len(timeSlots)):
    relationships += f'{timeSlots[ts1]}: '
    for ts2 in timeSlots:
        if timeSlots[ts1] != ts2:
            # Deals with same day slots.
            if timeSlots[ts1][:3] == ts2[:3]:
                relationships += f'{ts2} '
            # Deals with adjacent day slots.
            elif timeSlots[ts1][:3] == 'Mon' and ts2[:3] == 'Tue':
                relationships += f'{ts2} '
            elif timeSlots[ts1][:3] == 'Tue' and (ts2[:3] == 'Mon' or ts2[:3] == 'Wed'):
                relationships += f'{ts2} '
            elif timeSlots[ts1][:3] == 'Wed' and (ts2[:3] == 'Tue' or ts2[:3] == 'Thu'):
                relationships += f'{ts2} '
            elif timeSlots[ts1][:3] == 'Thu' and (ts2[:3] == 'Wed' or ts2[:3] == 'Fri'):
                relationships += f'{ts2} '
            elif timeSlots[ts1][:3] == 'Fri' and ts2[:3] == 'Thu':
                relationships += f'{ts2} '
    if ts1 != len(timeSlots) - 1:
        relationships += '; '

print(relationships)

neighbours = parse_neighbors(relationships)
neighbours

MonClass1: MonClass2 MonClass3 TueClass1 TueClass2 TueClass3 ; MonClass2: MonClass1 MonClass3 TueClass1 TueClass2 TueClass3 ; MonClass3: MonClass1 MonClass2 TueClass1 TueClass2 TueClass3 ; TueClass1: MonClass1 MonClass2 MonClass3 TueClass2 TueClass3 WedClass1 WedClass2 WedClass3 ; TueClass2: MonClass1 MonClass2 MonClass3 TueClass1 TueClass3 WedClass1 WedClass2 WedClass3 ; TueClass3: MonClass1 MonClass2 MonClass3 TueClass1 TueClass2 WedClass1 WedClass2 WedClass3 ; WedClass1: TueClass1 TueClass2 TueClass3 WedClass2 WedClass3 ThuClass1 ThuClass2 ThuClass3 ; WedClass2: TueClass1 TueClass2 TueClass3 WedClass1 WedClass3 ThuClass1 ThuClass2 ThuClass3 ; WedClass3: TueClass1 TueClass2 TueClass3 WedClass1 WedClass2 ThuClass1 ThuClass2 ThuClass3 ; ThuClass1: WedClass1 WedClass2 WedClass3 ThuClass2 ThuClass3 FriClass1 FriClass2 FriClass3 ; ThuClass2: WedClass1 WedClass2 WedClass3 ThuClass1 ThuClass3 FriClass1 FriClass2 FriClass3 ; ThuClass3: WedClass1 WedClass2 WedClass3 ThuClass1 ThuClass2 FriCla

defaultdict(list,
            {'MonClass1': ['MonClass2',
              'MonClass3',
              'TueClass1',
              'TueClass2',
              'TueClass3',
              'MonClass2',
              'MonClass3',
              'TueClass1',
              'TueClass2',
              'TueClass3'],
             'MonClass2': ['MonClass1',
              'MonClass1',
              'MonClass3',
              'TueClass1',
              'TueClass2',
              'TueClass3',
              'MonClass3',
              'TueClass1',
              'TueClass2',
              'TueClass3'],
             'MonClass3': ['MonClass1',
              'MonClass2',
              'MonClass1',
              'MonClass2',
              'TueClass1',
              'TueClass2',
              'TueClass3',
              'TueClass1',
              'TueClass2',
              'TueClass3'],
             'TueClass1': ['MonClass1',
              'MonClass2',
              'MonClass3',
              'MonClass1',
         

In [11]:
# CS2010: Practical Programming Methodology
# CS2040: Algorithms I
# CS3790: Operating Systems
# CS2910: Introduction to File and Database Management

courses = ['CS2010','CS2040','CS3790','CS2910']
types = ['Lec1','Lec2','Lab1','Lab2']
classes = []
for course in courses:
    for cT in types:
        if cT != 'Lab2':
            classes.append(course + cT)
        if course == 'CS2010' and cT == 'Lab2':
            classes.append(course + cT)
classes.append('FreeBlock1')
classes.append('FreeBlock2')

domains = {}
for slot in timeSlots:
    domains[slot] = classes

domains

{'MonClass1': ['CS2010Lec1',
  'CS2010Lec2',
  'CS2010Lab1',
  'CS2010Lab2',
  'CS2040Lec1',
  'CS2040Lec2',
  'CS2040Lab1',
  'CS3790Lec1',
  'CS3790Lec2',
  'CS3790Lab1',
  'CS2910Lec1',
  'CS2910Lec2',
  'CS2910Lab1',
  'FreeBlock1',
  'FreeBlock2'],
 'MonClass2': ['CS2010Lec1',
  'CS2010Lec2',
  'CS2010Lab1',
  'CS2010Lab2',
  'CS2040Lec1',
  'CS2040Lec2',
  'CS2040Lab1',
  'CS3790Lec1',
  'CS3790Lec2',
  'CS3790Lab1',
  'CS2910Lec1',
  'CS2910Lec2',
  'CS2910Lab1',
  'FreeBlock1',
  'FreeBlock2'],
 'MonClass3': ['CS2010Lec1',
  'CS2010Lec2',
  'CS2010Lab1',
  'CS2010Lab2',
  'CS2040Lec1',
  'CS2040Lec2',
  'CS2040Lab1',
  'CS3790Lec1',
  'CS3790Lec2',
  'CS3790Lab1',
  'CS2910Lec1',
  'CS2910Lec2',
  'CS2910Lab1',
  'FreeBlock1',
  'FreeBlock2'],
 'TueClass1': ['CS2010Lec1',
  'CS2010Lec2',
  'CS2010Lab1',
  'CS2010Lab2',
  'CS2040Lec1',
  'CS2040Lec2',
  'CS2040Lab1',
  'CS3790Lec1',
  'CS3790Lec2',
  'CS3790Lab1',
  'CS2910Lec1',
  'CS2910Lec2',
  'CS2910Lab1',
  'FreeBlock1',
 

In [12]:
def constraints(X, x, Y, y):
    # Deals with same classes.
    if x == y:
        return False
    # Deals with same course and class type.
    elif x[:9] == y[:9]:
        return False
    else:
        return True


# constraints = lambda X, x, Y, y: x != y

Task 2 - Developing CSP Implementation Based on CSP Class

In [13]:
from CSPclass import CSP

courseProblem = CSP(variables = None, domains = domains, neighbors = neighbours, constraints = constraints)

Task 3 - Applying the min_conflicts Algorithm to Solve the CSP

In [14]:
from algorithms import min_conflicts

schedule = min_conflicts(courseProblem)

var MonClass1 domain:
[('CS2010Lec1', 0), ('CS2010Lec2', 0), ('CS2010Lab1', 0), ('CS2010Lab2', 0), ('CS2040Lec1', 0), ('CS2040Lec2', 0), ('CS2040Lab1', 0), ('CS3790Lec1', 0), ('CS3790Lec2', 0), ('CS3790Lab1', 0), ('CS2910Lec1', 0), ('CS2910Lec2', 0), ('CS2910Lab1', 0), ('FreeBlock1', 0), ('FreeBlock2', 0)]
var MonClass2 domain:
[('CS2010Lec1', 0), ('CS2010Lec2', 0), ('CS2010Lab1', 0), ('CS2010Lab2', 0), ('CS2040Lec1', 0), ('CS2040Lec2', 0), ('CS2040Lab1', 0), ('CS3790Lec1', 0), ('CS3790Lec2', 0), ('CS3790Lab1', 0), ('CS2910Lec1', 2), ('CS2910Lec2', 2), ('CS2910Lab1', 0), ('FreeBlock1', 0), ('FreeBlock2', 0)]
var MonClass3 domain:
[('CS2010Lec1', 0), ('CS2010Lec2', 0), ('CS2010Lab1', 0), ('CS2010Lab2', 0), ('CS2040Lec1', 0), ('CS2040Lec2', 0), ('CS2040Lab1', 0), ('CS3790Lec1', 0), ('CS3790Lec2', 0), ('CS3790Lab1', 0), ('CS2910Lec1', 2), ('CS2910Lec2', 2), ('CS2910Lab1', 2), ('FreeBlock1', 0), ('FreeBlock2', 0)]
var TueClass1 domain:
[('CS2010Lec1', 0), ('CS2010Lec2', 0), ('CS2010Lab1', 

In [15]:
schedule

{'MonClass1': 'CS2910Lec2',
 'MonClass2': 'CS2910Lab1',
 'MonClass3': 'FreeBlock2',
 'TueClass1': 'CS2040Lec2',
 'TueClass2': 'CS2010Lab2',
 'TueClass3': 'CS2040Lab1',
 'WedClass1': 'CS3790Lab1',
 'WedClass2': 'CS3790Lec1',
 'WedClass3': 'CS2010Lec1',
 'ThuClass1': 'FreeBlock1',
 'ThuClass2': 'CS2040Lec1',
 'ThuClass3': 'CS2010Lab1',
 'FriClass1': 'CS2910Lec1',
 'FriClass2': 'CS3790Lec2',
 'FriClass3': 'CS2010Lec1'}

In [16]:
for c in range(-1, len(slots)):
    for d in range(-1, len(days)):
        if c == -1 and d == -1:
            print('          ', end='')
        elif c == -1:
            print(days[d], end='       ')
        elif d == -1:
            print(slots[c], end=' ')
        else:
            # print(schedule[days[d]+slots[c]][:10], end=' ')
            print(schedule[days[d]+slots[c]][:9], end=' ')
    print()

          Mon       Tue       Wed       Thu       Fri       
Class1 CS2910Lec CS2040Lec CS3790Lab FreeBlock CS2910Lec 
Class2 CS2910Lab CS2010Lab CS3790Lec CS2040Lec CS3790Lec 
Class3 FreeBlock CS2040Lab CS2010Lec CS2010Lab CS2010Lec 
